In [1]:
import pandas as pd
import numpy as np
import time

from nba_api.stats.endpoints import playercareerstats, scoreboard, commonplayerinfo, boxscoreplayertrackv2, boxscorematchups

from nba_api.stats.static import teams,players

# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season, SeasonType, SeasonAll
from nba_api.stats.endpoints import playbyplay
import pickle

In [2]:
season_list = []
for year in range(3):
    season_list.append('202'+str(year)+'-2'+str(year+1))

In [11]:
# Get all the NBA teams description
nba_teams = teams.get_teams()

status_data = np.array([])
season_campaign_all_teams_dict = {}
df_boxscores = pd.DataFrame()
count = 0
for selected_season in season_list:    
    season_campaign_all_teams_dict[selected_season] = {}
    for team_id in pd.DataFrame(nba_teams)['id']:
        time.sleep(1)
        campaign_all_teams_dict = {}
        
        campaign_all_teams_dict[team_id] = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id,
                                    season_nullable=selected_season,
                                    season_type_nullable=SeasonType.regular).get_data_frames()[0]

        campaign_all_teams_dict[team_id]['TEAM_ADVERSARY_ABBREVIATION'] = campaign_all_teams_dict[team_id]['MATCHUP'].apply(lambda x:x[-3:])
        campaign_all_teams_dict[team_id]['WINNER'] = campaign_all_teams_dict[team_id]['WL'].apply(lambda x:x == 'W')
        
        for game_id in campaign_all_teams_dict[team_id]['GAME_ID']:

            time.sleep(1)
            try:
                match_players_boxscore = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=game_id).get_data_frames()[0]
                selected_columns = ['GAME_ID','TEAM_ID','PLAYER_ID','PLAYER_NAME','START_POSITION']
                df = match_players_boxscore[selected_columns]
                df_boxscores = pd.concat([df_boxscores,match_players_boxscore[selected_columns]], ignore_index=True)
                print(f"{team_id} - {game_id} OK")
                status_data = np.append(status_data,[team_id,game_id,'OK'])                
            except:                
                print(f"{team_id} - {game_id} FAIL")
                status_data = np.append(status_data,[team_id,game_id,'FAIL'])
                time.sleep(10)

        season_campaign_all_teams_dict[selected_season][team_id] = campaign_all_teams_dict

1610612737 - 0022001066 OK
1610612737 - 0022001049 OK
1610612737 - 0022001042 OK
1610612737 - 0022001026 OK
1610612737 - 0022001000 OK
1610612737 - 0022000992 OK
1610612737 - 0022000978 OK
1610612737 - 0022000959 OK
1610612737 - 0022000949 OK
1610612737 - 0022000934 OK
1610612737 - 0022000916 OK
1610612737 - 0022000913 OK
1610612737 - 0022000898 OK
1610612737 - 0022000886 OK
1610612737 - 0022000876 OK
1610612737 - 0022000861 OK
1610612737 - 0022000840 OK
1610612737 - 0022000824 OK
1610612737 - 0022000805 OK
1610612737 - 0022000791 OK
1610612737 - 0022000780 OK
1610612737 - 0022000770 OK
1610612737 - 0022000759 OK
1610612737 - 0022000742 OK
1610612737 - 0022000735 OK
1610612737 - 0022000171 OK
1610612737 - 0022000705 OK
1610612737 - 0022000692 OK
1610612737 - 0022000677 OK
1610612737 - 0022000660 OK
1610612737 - 0022000640 OK
1610612737 - 0022000626 OK
1610612737 - 0022000613 OK
1610612737 - 0022000597 OK
1610612737 - 0022000590 OK
1610612737 - 0022000570 OK
1610612737 - 0022000544 OK
1

In [12]:
# df_boxscores.query("GAME_ID == '0022000465' and TEAM_ID == 1610612738").drop_duplicates()
df_boxscores_no_duplicates = df_boxscores.drop_duplicates()

df_boxscores_no_duplicates.to_csv('players_formation.csv')

with open('season_campaign_all_teams_dict.pickle','wb') as results:
    pickle.dump(season_campaign_all_teams_dict, results)

df_boxscores_no_duplicates.head()
# season_campaign_all_teams_dict['2020-21'][1610612740]
# df_boxscores.query("GAME_ID == '0022000465'")
# df_boxscores.query("TEAM_ID == 1610612738")
# df_boxscores['GAME_ID'].value_counts()

,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,START_POSITION
0,0022001066,1610612745,1630256,Jae'Sean Tate,F
1,0022001066,1610612745,1630231,Kenyon Martin Jr.,F
2,0022001066,1610612745,203482,Kelly Olynyk,C
3,0022001066,1610612745,1630237,Anthony Lamb,G
4,0022001066,1610612745,201571,D.J. Augustin,G


In [13]:

for y,d in season_campaign_all_teams_dict.items():
    for k,d1 in d.items():
        print(y,k)

2020-21 1610612737
2020-21 1610612738
2020-21 1610612739
2020-21 1610612740
2020-21 1610612741
2020-21 1610612742
2020-21 1610612743
2020-21 1610612744
2020-21 1610612745
2020-21 1610612746
2020-21 1610612747
2020-21 1610612748
2020-21 1610612749
2020-21 1610612750
2020-21 1610612751
2020-21 1610612752
2020-21 1610612753
2020-21 1610612754
2020-21 1610612755
2020-21 1610612756
2020-21 1610612757
2020-21 1610612758
2020-21 1610612759
2020-21 1610612760
2020-21 1610612761
2020-21 1610612762
2020-21 1610612763
2020-21 1610612764
2020-21 1610612765
2020-21 1610612766
2021-22 1610612737
2021-22 1610612738
2021-22 1610612739
2021-22 1610612740
2021-22 1610612741
2021-22 1610612742
2021-22 1610612743
2021-22 1610612744
2021-22 1610612745
2021-22 1610612746
2021-22 1610612747
2021-22 1610612748
2021-22 1610612749
2021-22 1610612750
2021-22 1610612751
2021-22 1610612752
2021-22 1610612753
2021-22 1610612754
2021-22 1610612755
2021-22 1610612756
2021-22 1610612757
2021-22 1610612758
2021-22 1610